In [ ]:
# Initial imports
import pandas as pd
from pathlib import Path

In [ ]:
#Data Loading
file_path = Path("crypto_data.csv")
df_crypto = pd.read_csv(file_path)
df_crypto.head()
df_crypto.shape

In [ ]:
#Eliminate crypto currencies that don't trade
df_crypto2 = df_crypto[df_crypto.IsTrading == True]
df_crypto2.shape

In [ ]:
#drop the IsTrading column
df_crypto3 = df_crypto2.drop(columns=["IsTrading"])
df_crypto3.shape

In [ ]:
#Drop any rows with null values
df_crypto4 = df_crypto3.dropna()
df_crypto4.shape

In [ ]:

#Drop the unneccesary columns
df_crypto4.head()
df_crypto5 = df_crypto4.drop(columns=["CoinName","Unnamed: 0"])
df_crypto5.shape
df_crypto5.TotalCoinSupply = df_crypto5.TotalCoinSupply.astype(float)
df_crypto5.dtypes

In [ ]:
# create dummy variables for categorized columns
df_crypto6 = pd.get_dummies(df_crypto5)
df_crypto6.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

pca = PCA(n_components=.90)

df_crypto6_scaled = StandardScaler().fit_transform(df_crypto6)

crypto6_pca = pca.fit_transform(df_crypto6_scaled)

df_crypto6_pca = pd.DataFrame(data=crypto6_pca)

df_crypto6_pca

In [ ]:
from sklearn.manifold import TSNE

# Initialize t-SNE model
tsne = TSNE(learning_rate=35)

# Reduce dimensions
tsne_features = tsne.fit_transform(crypto6_pca)

In [ ]:
tsne_features.shape

In [ ]:
tsne_df = pd.DataFrame(tsne_features, columns=['x','y'])
tsne_df

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(tsne_df['x'], tsne_df['y'])
plt.show()

In [ ]:
import numpy as np
from sklearn.cluster import KMeans

inertia = []
# Same as k = list(range(1, 11))
k = [1,2,3,4,5,6,7,8,9,10]

# Looking for the best k
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(tsne_df)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

plt.plot(df_elbow['k'], df_elbow['inertia'])
plt.xticks(range(1,11))
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()